## Day 4

In [1]:
import numpy as np
import torch



In [2]:
def parallel_db(db, remove_entry):
    return torch.cat((db[:remove_entry], db[remove_entry + 1:]))
    

In [3]:
# parallel_db(db, remove_entry).shape

In [4]:
def get_parallel_dbs(db):
    
    parallel_dbs = []

    for i in range(len(db)):
        parallel_dbs.append((parallel_db(db, i)))
        
    return parallel_dbs

In [5]:
 def create_db_and_parallel(num_entries):
#         num_entries = 5000
        db = torch.randn(num_entries) > 0.5
        
        pdbs = get_parallel_dbs(db)
        
        return db, pdbs
    

In [6]:
def query(db):
#     return db.sum()
    return db.float().mean() #Gives emperical sensitivity

> Find difference between db.sum() and sum(db)

In [7]:
def sensitivity_fn(query, num_entries = 500):
    
    db, pdbs = create_db_and_parallel(num_entries)
    
    full_db_result = query(db)
    print(full_db_result)
    
    sensitivity = 0

    for pdb in pdbs:
        pdb_result = query(pdb)
        db_distance = torch.abs(pdb_result - full_db_result)

        if (db_distance > sensitivity):
            sensitivity = db_distance
            
    return sensitivity

In [8]:
sensitivity_fn(query)

tensor(0.2860)


tensor(0.0014)

In [9]:
# while removing/ calculating parallel db, each entry corresponds to the unique person... 

In [10]:
def query(db, threshold=5):
    return (db.sum() > threshold).float() #L1 sensitivity

In [11]:
db, queries = create_db_and_parallel(10)
print(query(db), db.sum())

tensor(0.) tensor(3)


In [12]:
for i in range(10):
    sens_f = sensitivity_fn(query, num_entries = 100)
    print(sens_f)

tensor(1.)
0
tensor(1.)
0
tensor(1.)
0
tensor(1.)
0
tensor(1.)
0
tensor(1.)
0
tensor(1.)
0
tensor(1.)
0
tensor(1.)
0
tensor(1.)
0


# perform defferencing attack on row 10

In [13]:
db, _ = create_db_and_parallel(100)
print(db, db[10])

tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
        0, 0, 1, 1], dtype=torch.uint8) tensor(0, dtype=torch.uint8)


In [14]:
pdb = parallel_db(db, 10)

In [15]:
db[10]

tensor(0, dtype=torch.uint8)

In [16]:
#differencing attack using sum
db.sum() - pdb.sum()

tensor(0)

In [17]:
#differencing attack using mean

(sum(db).float() / len(db)) - (sum(pdb).float() / len(pdb))

tensor(-0.0023)